<a href="https://colab.research.google.com/github/aparna-2001/Leukemia_GenePatterns/blob/main/dataset_leukemia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**What is my project aim?**

- To analyse the gene expression data and identify patterns that distinguish ALL(Acute Lyphoid Leukemia) and AML(Acute myeloid leukemia)using matrix methods, PCA, ANOVA, Linear regression, graph analysis in python.

In [ ]:
#pipleine for analysis
   #file handling
   #Data structuring
   #Data cleaning and pre processing
   #Exploratory data analysis
   #PCA
   #ANOVA
   #linear regression
   #Graph analysis
   #interpretation
   #visualisation

In [ ]:
#converting the data file to table and then to matrix


import zipfile
import pandas as pd
import numpy as np

with zipfile.ZipFile("archive.zip",'r') as zf:
  with zf.open("data_set_ALL_AML_independent.csv") as file:
    data_table = pd.read_csv(file)

#data_matrix = np.array(data_table)

print(data_table.head())
#print(data_matrix)
#print(data_matrix.shape)




                      Gene Description Gene Accession Number   39 call   40  \
0  AFFX-BioB-5_at (endogenous control)        AFFX-BioB-5_at -342    A  -87   
1  AFFX-BioB-M_at (endogenous control)        AFFX-BioB-M_at -200    A -248   
2  AFFX-BioB-3_at (endogenous control)        AFFX-BioB-3_at   41    A  262   
3  AFFX-BioC-5_at (endogenous control)        AFFX-BioC-5_at  328    A  295   
4  AFFX-BioC-3_at (endogenous control)        AFFX-BioC-3_at -224    A -226   

  call.1   42 call.2   47 call.3  ...   65 call.29   66 call.30   63 call.31  \
0      A   22      A -243      A  ...  -62       A  -58       A -161       A   
1      A -153      A -218      A  ... -198       A -217       A -215       A   
2      A   17      A -163      A  ...   -5       A   63       A  -46       A   
3      A  276      A  182      A  ...  141       A   95       A  146       A   
4      A -211      A -289      A  ... -256       A -191       A -172       A   

    64 call.32   62 call.33  
0  -48       A

In [ ]:
#filtering the columns which only contains numeric values

numeric_cols = data_table.select_dtypes(include="number").columns
#print(numeric_cols)

#creating new table with only numeric data
data_numeric = data_table[numeric_cols]
#print(data_numeric)

data_numeric_matrix = np.array(data_numeric)

##checking the data
print(data_numeric.head())
#print(data_numeric_matrix.shape)
#print(data_numeric.describe())

#mean




    39   40   42   47   48   49   41   43   44   45  ...   54   57   58   60  \
0 -342  -87   22 -243 -130 -256  -62   86 -146 -187  ...  -90 -137 -157 -172   
1 -200 -248 -153 -218 -177 -249  -23  -36  -74 -187  ...  -87  -51 -370 -122   
2   41  262   17 -163  -28 -410   -7 -141  170  312  ...  102  -82  -77   38   
3  328  295  276  182  266   24  142  252  174  142  ...  319  178  340   31   
4 -224 -226 -211 -289 -170 -535 -233 -201  -32  114  ... -283 -135 -438 -201   

    61   65   66   63   64   62  
0  -47  -62  -58 -161  -48 -176  
1 -442 -198 -217 -215 -531 -284  
2  -21   -5   63  -46 -124  -81  
3  396  141   95  146  431    9  
4 -351 -256 -191 -172 -496 -294  

[5 rows x 34 columns]


In [ ]:
# checking for missing values

print(data_numeric.isnull().values.any())
print(data_numeric.isnull().sum())

False
39    0
40    0
42    0
47    0
48    0
49    0
41    0
43    0
44    0
45    0
46    0
70    0
71    0
72    0
68    0
69    0
67    0
55    0
56    0
59    0
52    0
53    0
51    0
50    0
54    0
57    0
58    0
60    0
61    0
65    0
66    0
63    0
64    0
62    0
dtype: int64


- since there are no missing values wecan proceed further with statistical analysis

In [ ]:
#just checking for the type

print(type(data_numeric))
print(type(data_matrix))

<class 'pandas.core.frame.DataFrame'>


NameError: name 'data_matrix' is not defined

In [ ]:
#basic statistics of the data

#print(data_numeric.describe())


#mean across the data
gene_mean = data_numeric.mean(axis = 1)
#print(gene_mean)
avg_mean = gene_mean.mean() #avg expression of one gene across all data
print(f"The average mean: {avg_mean}")


#variation across the data
gene_std_dev = data_numeric.std(axis = 1)
avg_std_dev = gene_std_dev.mean()
print(f"The average standard deviation: {avg_std_dev}")

#using median per patient
#print(data_numeric.shape)

- The average mean of the data set is approximately 599.75,
    - which mean the mean value of gene expression of all genes in patient is around 599.79

- The standard deviation for the gene expression across all patients is 394.23.
   -This high variation suggests that some genes are highly expressed while others are lowly expressed, which is typical in gene expression datasets.


caution: This value can only help you in gene expression studies. But this won't give you an absolute picture. because the data values strtches to both extremes(high negative to high positive)

In [ ]:
#Now let's do PCA.

   #PCA is the method of reducing dimensions
      #that is our aim here is to reduce the gene expressions number
           #we have 7129 genes across 34 patients after patients
               #we try to find the pattern by using lesser number of genes

In [ ]:
#PCA - principal component analysis

data_numeric_transpose = data_numeric.T
#print(data_numeric_transpose)

#step1: conversion to matrix
data_numeric_transpose_matrix = np.array(data_numeric_transpose)
#print(type(data_numeric_transpose_matrix))

#step2:mean of each column
data_numeric_transpose_mean = data_numeric_transpose.mean(axis=0)
#print(data_numeric_transpose_mean)

#another way for finding the mean of each column
#count = 0
#for i in range(data_numeric_transpose_matrix.shape[1]):
#  col_mean = np.mean(data_numeric_transpose_matrix[:,i])
#  print(col_mean)
#  count += 1
#  if count == 5:
#    break

#step3: subtracting the mean from each column generatin a mean centered matrix
mean_centered_matrix = data_numeric_transpose_matrix - data_numeric_transpose_mean.values

#step4: finding covariance matrix
cov_matrix = np.cov(mean_centered_matrix,rowvar = False)
print(cov_matrix)


#step5: finding the eigen values and eigen vectors
eigen_values, eigen_vectors = np.linalg.eig(cov_matrix)


[[ 6.99934492e+03  2.19050891e+03  6.37202317e+02 ... -1.49815508e+02
   1.50960446e+04  1.35949465e+03]
 [ 2.19050891e+03  1.32292201e+04  2.53110784e+03 ...  3.09485740e+02
  -7.70641711e+02  1.26540731e+03]
 [ 6.37202317e+02  2.53110784e+03  1.66603859e+04 ... -3.37498128e+03
  -6.87090018e+03 -4.90416221e+02]
 ...
 [-1.49815508e+02  3.09485740e+02 -3.37498128e+03 ...  3.01738057e+03
   9.13989305e+02  9.69981283e+02]
 [ 1.50960446e+04 -7.70641711e+02 -6.87090018e+03 ...  9.13989305e+02
   2.29828185e+05 -2.48652406e+03]
 [ 1.35949465e+03  1.26540731e+03 -4.90416221e+02 ...  9.69981283e+02
  -2.48652406e+03  2.40396168e+03]]
